In [ ]:
import matplotlib.pyplot as plt
import numpy as np

import lsst.daf.butler as dB
import lsst.cp.verify.notebooks.utils as utils
import lsst.afw.display as afwDisplay

In [ ]:
# This cell contains parameters that can be automatically set via the papermill package.
# Examples:
#   papermill --prepare-only -p calibType newBias -p cameraName LSSTCam <input> <output>
#   papermill -p interactive False <input> <output>
interactive = True

# Which repository to use.
repository = '/repo/embargo/'

# Which calibration type to analyse.
calibType = 'linearity'
detectorNumber = 0

# Which camera the calibration is for.
cameraName = 'LATISS'

# Which display to use.
displayBackend = 'astrowidgets'

# Which collections the calibration was constructed in.
genCollection = 'u/czw/DM-37819/linearity_v25.20230321a'
genCollection2 = 'u/czw/DM-37819/linearity_v26.20230321a'

# Which collections containing the PTC objects.
ptcCollection = 'u/czw/DM-38111/ptc-v25.20230222a'
ptcCollection2 = 'u/czw/DM-38111/ptc-v26.20230222a'

# Which collections containing the verification outputs.
verifyCollection = 'u/czw/DM-37819/linearityVerify_v25.20230321a'
verifyCollection2 = 'u/czw/DM-37819/linearityVerify_v26.20230321a'

In [ ]:
# Get butler and camera
butler = dB.Butler(repository)
camera = butler.get('camera', instrument=cameraName, collections='LATISS/calib')

In [ ]:
linA = butler.get('linearity', instrument=cameraName, detector=detectorNumber, 
                  collections=genCollection)
linB = butler.get('linearity', instrument=cameraName, detector=detectorNumber, 
                  collections=genCollection2)

In [ ]:
ptcA = butler.get('ptc', instrument=cameraName, detector=detectorNumber, 
                  collections=ptcCollection)
ptcB = butler.get('ptc', instrument=cameraName, detector=detectorNumber, 
                  collections=ptcCollection2)

In [ ]:
offset = 0
fig, axes = plt.subplots(nrows=1, ncols=2,
                         sharex=False, sharey=False, figsize=[8.0, 8.0 / 2], 
                         constrained_layout=True)
for it, amp in enumerate(camera[0].getAmplifiers()):
    if linA.linearityType[amp.getName()] == 'Spline':
        centers, values = np.split(linA.linearityCoeffs[amp.getName()], 2)
        axes[0].scatter(centers, values + it * offset, marker='+', label="LinA")
        
        centers, values = np.split(vlinA.linearityCoeffs[amp.getName()], 2)
        axes[0].scatter(centers, values + it * offset, marker='+', label="VerifyA")

    if linB.linearityType[amp.getName()] == 'Spline':
        centers, values = np.split(linB.linearityCoeffs[amp.getName()], 2)
        axes[1].scatter(centers, values + it * offset, marker='x', label='LinB')

        centers, values = np.split(vlinB.linearityCoeffs[amp.getName()], 2)
        axes[1].scatter(centers, values + it * offset, marker='+', label="VerifyB")
axes[1].axline((55000, 50), slope=-1e-2, color='cyan')
axes[0].set_title("v25")
axes[1].set_title("v26")
plt.show()

In [ ]:
import lsst.afw.math as afwMath

continueDisplay=True
for amp in ptcA.ampNames:
    if linA.linearityType[amp] == 'Spline':
        centers, values = np.split(linA.linearityCoeffs[amp], 2)
        interp = afwMath.makeInterpolate(centers.tolist(), values.tolist(),
                                         afwMath.stringToInterpStyle("AKIMA_SPLINE"))
        delta = interp.interpolate(np.array(ptcA.rawMeans[amp]))
        linearized = np.array(ptcA.rawMeans[amp]) - np.array(delta)
    
    plt.scatter(linearized, ptcA.rawVars[amp], marker='o', label="PTCA")

    plt.scatter(ptcA.rawMeans[amp], ptcA.rawVars[amp], marker='+', label="PTCA")
    plt.scatter(ptcB.rawMeans[amp], ptcB.rawVars[amp], marker='x', label='PTCB')
    plt.show()

In [ ]:
plt.scatter(np.array(list(ptcA.gain.values())), 
            np.array(list(ptcA.gain.values())) - np.array(list(ptcB.gain.values())), 
            marker='+', label="PTCA")
plt.scatter(np.array(list(ptcA.noise.values())), 
            np.array(list(ptcA.noise.values())) - np.array(list(ptcB.noise.values())),
            marker='x', label='PTCB')
plt.show()
print("A Gain:", [f"{x:.3f}" for x in ptcA.gain.values()])
print("A RN:  ", [f"{x:.3f}" for x in ptcA.noise.values()])

print("B Gain:", [f"{x:.3f}" for x in ptcB.gain.values()])
print("B RN:  ", [f"{x:.3f}" for x in ptcB.noise.values()])